In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc 
import dash_bootstrap_components as dbc 

from dash.dependencies import Output, Input

import pandas as pd
import numpy as np
import sqlalchemy

from datetime import datetime
import time
import ta

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


In [ ]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@app.callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run_server(debug=True)


In [16]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3, 4, 5, 6, 7, 8],
    y=[0, 1, 2, 3, 4, 5, 6, 7, 8]
))

fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

In [23]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3, 4, 5, 6, 7, 8],
    y=[0, 1, 2, 3, 4, 5, 6, 7, 8]
))

fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

In [ ]:

# ----------------------- CPPI ------------------------------
@app.callback(
    Output(component_id='cppi', component_property='figure'),
    Input('interval-component', 'n_intervals')
)
def update_graph_current_histogram(n):
    Trades = merge("Trades", "TradesMeta")
    
    trace = go.Scatter(
        x = Trades.index ,
        y = Trades["returns"],
        line = dict(color = 'blue' , width = 1.8),
        name = 'returns'
        )
    
    layout = go.Layout(
        height = 300 , width = 500,
        template="simple_white"
        )
    
    fig = go.Figure(
        data = [trace],
        layout = layout
    )    
    return fig



In [ ]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [5, 6, 7, 8]})

In [ ]:
df1

In [ ]:
df2

In [ ]:
df = Data("Trades").merge(Data("TradesMeta"), on = "identifier")

In [ ]:
app = dash.Dash(__name__, title = "cedTradAnalytics")

## figure

In [ ]:
fig_0 = subplots(nb_rows = 1 , nb_cols = 2)

fig_0.add_trace(
    go.Scatter(
        x = trade.index,
        y = trade["cum_ret"]
    )
)

In [ ]:
fig = subplots(nb_rows = 1 , nb_cols = 3)

# Recovery
fig.add_trace(
    go.Scatter(
        x = Benchmark.index,
        y = Benchmark["price_return"],
        name = "recovery"
    ),
    col = 1, row = 1
)

# Price
fig.add_trace(
    go.Scatter(
        x = Benchmark.index,
        y = Benchmark["price"],
        name = "price"
    ),
    col = 2, row = 1
)

plot_LSB(fig, col = 3 , row = 1 ,Benchmark = R.Trades, Trades = R.Trades_reduit,
             Trades_long = R.Trades_reduit_long, Trades_short = R.Trades_reduit_short)

Fig = go.FigureWidget(fig)

## layout_0

In [ ]:
layout_0 = html.Div([
    html.Div([
        
        html.Div([
            dcc.Graph(id = 'current_trade', figure = fig_0)
        ]),
        html.Div([
            dcc.Graph(id = "cppi", figure = fig_0)
        ])
    ],
             style = {'display' : 'inline-block', 'width' : '58%'}),
    
    html.Div([
        html.H3("Current trade info"),
        html.Br(),
        html.H4("Entry"),
        html.Div(id = "price"),
        html.Div(id = "amount"),
        html.Div(id = "units"),
        html.Br(),
        html.H4("Current"),
        html.Div(id = "price_c"),
        html.Div(id = "amount_c"),
        html.Div(id = "cum_ret")
    ],
             style = {'display' : 'inline-block', 'width' : '38%'})
])

## layout_1

In [ ]:
layout_1 = html.Div([
    html.Br(),
    dcc.Graph( figure = Fig)
])


In [ ]:
layout_2 = html.Div([
    # Stats_ return
    html.Div([
        html.Div(id = "cum_ret"),
        html.Div(id = "drawdown"),
        html.Div(id = "min_ret"),
        html.Div(id = "max_ret"),
        html.Div(id = "mean"),
        html.Div(id = "VaR"),
        html.Div(id = "CVaR")
    ]),
    # stats 
    html.Div([
        html.Div(id = "amount"),
        html.Div(id = "amount_win"),
        html.Div(id = "amount_loss"),
        html.Div(id = "win_rate"),
        html.Div(id = "sharpe_ratio"),
        html.Div(id = "Expectancy"),
    ]),
])

In [ ]:
app.layout = html.Div([
    layout_0,
    layout_1
])

In [ ]:
if __name__ == '__main__':
    app.run_server(debug = False)

In [ ]:
@app.callback(
    Output(component_id='current_trade', component_property='figure'),
    Input('interval-component', 'n_intervals')
)
def update_graph_current_trade(n):
    
    Trades = Data(symbol = symbol, table = "Trades")
    loc = np.where(Trades["trade_id"] == Trades["trade_id"].iloc[-1])
    current_trade = Trades.iloc[loc]
    
    trace = go.Scatter(
        x = current_trade.index ,
        y = current_trade["cum_ret"],
        line = dict(color = 'blue' , width = 1.8),
        name = 'current trade'
        )
    
    layout = go.Layout(
        height = 300 , width = 500,
        template="simple_white"
        )
    
    fig = go.Figure(
        data = [trace],
        layout = layout
    )    
    return fig

## Test

In [ ]:

import pandas as pd
import numpy as np
import sqlalchemy
import datetime

import dash
import dash_core_components as dcc
import dash_html_components as html

import plotly.express as px
import plotly.graph_objects as go

from dash.dependencies import Input, Output


def Data(symbol, table):
    engine = sqlalchemy.create_engine(f'sqlite:///base_{symbol}.db')
    data = pd.read_sql(table, engine)
    data['date'] = pd.to_datetime(data['date'])
    data.set_index("date", inplace=True)
    data["cum_ret"] = data["cum_ret"].astype("float64")
    return data

symbol = "BTC"


app = dash.Dash(__name__)

app.layout = html.Div([
    
    html.Div([
        dcc.Slider(min = 0.5, max = 5,
                   step = 0.5, value = 1,
                   id = "interval-refresh")
    ],
             style = {"width" : "20%"}),
    
    html.Div(id='latest-timestamp', style={"padding": "20px"}),
    dcc.Interval(
            id='interval-component',
            interval=1 * 1000,
            n_intervals=0
    ),
    
    html.Div([
        dcc.Graph(id = "main")
    ])
])


@app.callback(
    [Output(component_id = "interval-component", component_property = "interval")],
    [Input("interval-refresh", "value")]
)
def update_refresh_rate(value):
    return [value * 1000]


@app.callback(
    [Output(component_id='latest-timestamp', component_property='children')],
    [Input('interval-component', 'n_intervals')]
)
def update_timestamp(interval):
    return [html.Span(f"Last updated: {datetime.datetime.now()}")]

@app.callback(
    Output(component_id = "main", component_property = "figure"),
    Input("interval-component", "n_intervals")
)
def update_graph(n):
    Trades = Data(symbol = symbol, table = "Trades")
    Trades_reduit = Data(symbol = symbol, table = "Trades_reduit")
    Long = Data(symbol = symbol, table = "Trades_long")
    Short = Data(symbol = symbol, table = "Trades_short")
    
    
    trades_line = go.Scatter(
        x = Trades.index ,
        y = Trades["cum_ret"],
        line = dict(color = 'blue' , width = 1.8),
        name = 'strategie'
        )
    
    price_line = go.Scatter(
        x = Trades.index ,
        y = Trades["price_cum_ret"],
        line = dict(color = 'black' , width = 1.8),
        name = 'price'
        )
    
    long_line = go.Scatter(
        x = Long.index ,
        y = Long["cum_ret"], 
        line = dict(color = 'green' , width = 1.8),
        name = 'long_line'
        )
    
    short_line = go.Scatter(
        x = Short.index ,
        y = Short["cum_ret"], 
        line = dict(color = 'red' , width = 1.8),
        name = 'short_line'
        )
    
    layout = go.Layout(
        height = 600 , width = 1500,
        template="simple_white"
        )
    
    fig = go.Figure(
        data = [trades_line, price_line, long_line, short_line],
        layout = layout
    )    
    return fig


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

# Graphe

In [ ]:
import plotly.express as px
df = px.data.medals_long()

fig = px.bar(df, x="medal", y="count", color="nation", text_auto=True)
fig.show()

In [ ]:
import plotly.express as px
df = px.data.medals_long()

fig = px.bar(df, x="medal", y="count", color="nation",
             pattern_shape="nation", pattern_shape_sequence=[".", "x", "+"])
fig.show()

In [ ]:
import plotly.graph_objects as go
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(name='SF Zoo', x=animals, y=[20, 14, 23]),
    go.Bar(name='LA Zoo', x=animals, y=[12, 18, 29])
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
import plotly.graph_objects as go
x = [1, 2, 3, 4]

fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=[1, 4, 9, 16]))
fig.add_trace(go.Bar(x=x, y=[6, -8, -4.5, 8]))
fig.add_trace(go.Bar(x=x, y=[-15, -3, 4.5, -8]))
fig.add_trace(go.Bar(x=x, y=[-1, 3, -3, -4]))

fig.update_layout(barmode='relative', title_text='Relative Barmode')
fig.show()

In [ ]:
import plotly.graph_objects as go

animals = ['giraffes']

trace_1 = go.Bar(
    name = 'SF Zoo', x = animals , y = [20, 14, 23], text = animals
)

trace_2 = go.Bar(
    name = 'LA Zoo', x = animals , y = [12, 18, 29], text = animals
)

fig = go.Figure(
    data = [trace_1, trace_2]
)
fig.update_layout(barmode = 'stack')

#fig.update_xaxes(range= [0, 3])
#fig.update_yaxes(range = [0, 100])

fig

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = [1.5, 4.5],
        y = [0.75, 0.75],
        text = ["Unfilled Rectangle", "Filled Rectangle"],
        mode = "text"
    )
)

# set axes properties
fig.update_xaxes(range = [0, 7], showgrid = False)
fig.update_yaxes(range = [0, 3.5])

# Add shapes
fig.add_shape(type = "rect", 
              x0 = 1, y0 = 1, x1 = 2, y1 = 3,
              line = dict(color = "RoyalBlue"))

fig.add_shape(type = "rect", 
              x0 = 3, y0 = 1, x1 = 6, y1 = 2,
              line = dict(color = "RoyalBlue", width = 2),
              fillcolor = "LightSkyBlue")

fig.update_shapes(dict(xref = 'x', yref = 'y'))

fig

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Create scatter trace of text labels
fig.add_trace(go.Scatter(
    x=[1.5, 3],
    y=[2.5, 2.5],
    text=["Rectangle reference to the plot",
          "Rectangle reference to the axes"],
    mode="text",
))

# Set axes properties
fig.update_xaxes(range=[0, 4])
fig.update_yaxes(range=[0, 4])

# Add shape
fig.add_shape(type = "rect",
              xref = "x", yref = "y",
              x0 = 2.5, y0 = 0,
              x1 = 3.5, y1 = 2,
              line = dict(
                  color = "RoyalBlue",
                  width = 3
              ),
              fillcolor="LightSkyBlue")



In [ ]:
Fig = go.FigureWidget(fig)
Fig.update_layout(height = 800 , width =1500,
                  margin=dict(
                        l=5, r=0, b=10, t=30,
                        pad=1
                    ),
                  showlegend=True,
                  )
Fig.update_layout(legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.02,
                        xanchor="right",
                        x=0.7
                    )
                  )

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Add scatter trace for line
fig.add_trace(go.Scatter(
    x=["2015-02-01", "2015-02-02", "2015-02-03", "2015-02-04", "2015-02-05",
       "2015-02-06", "2015-02-07", "2015-02-08", "2015-02-09", "2015-02-10",
       "2015-02-11", "2015-02-12", "2015-02-13", "2015-02-14", "2015-02-15",
       "2015-02-16", "2015-02-17", "2015-02-18", "2015-02-19", "2015-02-20",
       "2015-02-21", "2015-02-22", "2015-02-23", "2015-02-24", "2015-02-25",
       "2015-02-26", "2015-02-27", "2015-02-28"],
    y=[-14, -17, -8, -4, -7, -10, -12, -14, -12, -7, -11, -7, -18, -14, -14,
       -16, -13, -7, -8, -14, -8, -3, -9, -9, -4, -13, -9, -6],
    mode="lines",
    name="temperature"
))

# Add shape regions
fig.add_vrect(
    x0="2015-02-04", x1="2015-02-06",
    fillcolor="LightSalmon", opacity=0.5,
    layer="below", line_width=0,
),

fig.add_vrect(
    x0="2015-02-20", x1="2015-02-22",
    fillcolor="LightSalmon", opacity=0.5,
    layer="below", line_width=0,
)

fig.show()

In [ ]:
import plotly.graph_objects as go

labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
values = [4500, 2500, 1053, 500]

Pie_chart = go.Pie(
    labels = labels, values = values
)

fig = go.Figure(
    data = [Pie_chart]
)

fig

In [ ]:
import plotly.graph_objects as go

labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
values = [4500, 2500, 1053, 500]

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
fig.show()

In [ ]:
import plotly.graph_objects as go

fig =go.Figure(go.Sunburst(
    labels=["Eve", "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
    parents=["", "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve" ],
    values=[10, 14, 12, 10, 2, 6, 6, 4, 4],
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame([
    dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', Resource="Max")
])

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Resource")
fig.show()

In [ ]:
import plotly.express as px

df = px.data.gapminder().query("year == 2007")
fig = px.sunburst(df, path=['continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'])
fig.show()

In [ ]:
import plotly.express as px
data = dict(
    number=[39, 27.4, 20.6, 11, 2],
    stage=["Website visit", "Downloads", "Potential customers", "Requested price", "Invoice sent"])
fig = px.funnel(data, x='number', y='stage')
fig.show()

In [ ]:
import plotly.graph_objects as go
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen'],
                             values=[4500,2500,1053,500])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()
